In [ ]:
import onnxruntime
import onnx
from onnx2keras import onnx_to_keras
from onnx_pytorch import code_gen
import numpy as np
import torch
from foolbox import PyTorchModel
from foolbox.attacks import LinfPGD
import tensorflow as tf
from art.attacks.evasion import FastGradientMethod
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from advbox import FGSM

In [ ]:
def sigmoid(input):
    return 1 / (1 + np.exp(-input))

In [ ]:
def onnx2Keras(path):
    onnx_model = onnx.load(path)
    model = onnx_to_keras(onnx_model, ['input_imgs', 'big_input_imgs', 'desire', 'traffic_convention', 'initial_state'],
                          name_policy='renumerate', change_ordering=False)
    model.save("models/keras")

In [ ]:
onnx_model = "models/supercombo.onnx"
session = onnxruntime.InferenceSession(onnx_model, None)
kerasModel = tf.keras.models.load_model("models/keras", compile=False)
pytorchModel = torch.load('models/supercombo.pt')
pytorchModel.eval()

In [ ]:
dataset = np.load('data/numpy.npz')

datasize = dataset['inputImgs'].shape[0]

inputImgs_data = dataset['inputImgs']
bigInputImgs_data = dataset['bigInputImgs']
desire_data = dataset['desire']
trafficConvention_data = dataset['trafficConvention']
initialState_data = dataset['initialState']
output_data = dataset['output']

plan_start_idx = 0
plan_end_idx = 4955

lanes_start_idx = plan_end_idx
lanes_end_idx = lanes_start_idx + 528

lane_lines_prob_start_idx = lanes_end_idx
lane_lines_prob_end_idx = lane_lines_prob_start_idx + 8

In [ ]:
def runModel(data):
    results = None
    for x in range(datasize):
        result = session.run([session.get_outputs()[0].name], {
            session.get_inputs()[0].name: np.vsplit(data['inputImgs'], datasize)[x],
            session.get_inputs()[1].name: np.vsplit(data['bigInputImgs'], datasize)[x],
            session.get_inputs()[2].name: np.vsplit(data['desire'], datasize)[x],
            session.get_inputs()[3].name: np.vsplit(data['trafficConvention'], datasize)[x],
            session.get_inputs()[4].name: np.vsplit(data['initialState'], datasize)[x],
        })
        result = result[0]
        lane_lines_prob = result[:, lane_lines_prob_start_idx:lane_lines_prob_end_idx:2]
        lane_lines_prob = sigmoid(lane_lines_prob)
        if np.any(results):
            results = np.concatenate((results, lane_lines_prob), axis=0)
        else:
            results = lane_lines_prob
    probabilityPoint = 0.7
    probability = (results >= probabilityPoint).astype(int)
    accuracy = np.sum(probability == output_data) /\
               (len(output_data) * len(output_data[0]))
    return accuracy

Hieronder zullen alle tools getest worden.

Testing tool –Adversial Robustness 360 Toolbox–

In [ ]:
try:
    estimator = ImageSegmentation(model=kerasModel, use_logits=False)
    attack = FastGradientMethod(estimator=estimator, eps=0.2)
    x_test_adv = attack.generate(x={"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data})
    accuracy = runModel(x_test_adv)
    print("Accuracy on adversarial test examples: {}%".format(accuracy))
except Exception as e: print("Image Segmentation does not exist")

Testing tool –Cleverhans–

In [ ]:
try:
    x_fgm = fast_gradient_method(kerasModel, {"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, 0.2, np.inf)
    accuracy = runModel(x_fgm)
    print("Accuracy on FGM adversarial examples:{}%".format(accuracy))
except Exception as e: print("Unsupported type dict expected Tensor")

Testing tool –Foolbox–

In [ ]:
try:
    model = "model/supercombo.pt"
    fmodel = PyTorchModel(model, bounds=(0, 1))

    attack = LinfPGD()
    _, advs, success = attack(fmodel, {"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, epsilons=[0.2])
    accuracy = runModel(advs)
    print("Accuracy on FGM adversarial examples:{}%".format(accuracy))
except Exception as e: print("Attack expected Images and Labels")

Testing tool –AdversarialBox–

In [ ]:
try:
    attack = FGSM(kerasModel)
    attack_config = {"epsilons": 1, "epsilons_max": 10, "epsilon_steps": 1, "steps": 100}
    adversary = attack({"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, **attack_config)
except Exception as e: print("Attack expected an image but got a Dict")

In [ ]:
try:
    model = "model/supercombo.pt"
    fmodel = PyTorchModel(model, bounds=(0, 1))

    attack = LinfPGD()
    _, advs, success = attack(fmodel, {"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, epsilons=[0.2])
    accuracy = runModel(advs)
    print("Accuracy on FGM adversarial examples:{}%".format(accuracy))
except Exception as e: print("Attack expected Images and Labels")

Testing tool –AdversarialBox–

In [ ]:
try:
    attack = FGSM(kerasModel)
    attack_config = {"epsilons": 1, "epsilons_max": 10, "epsilon_steps": 1, "steps": 100}
    adversary = attack({"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, **attack_config)
except Exception as e: print("Attack expected an image but got a Dict")

In [11]:
try:
    model = "model/supercombo.pt"
    fmodel = PyTorchModel(model, bounds=(0, 1))

    attack = LinfPGD()
    _, advs, success = attack(fmodel, {"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, epsilons=[0.2])
    accuracy = runModel(advs)
    print("Accuracy on FGM adversarial examples:{}%".format(accuracy))
except Exception as e: print("Attack expected Images and Labels")

ValueError: expected model to be a torch.nn.Module instance

Testing tool –AdversarialBox–

In [ ]:
try:
    attack = FGSM(kerasModel)
    attack_config = {"epsilons": 1, "epsilons_max": 10, "epsilon_steps": 1, "steps": 100}
    adversary = attack({"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data}, **attack_config)
except Exception as e: print("Attack expected an image but got a Dict")